In [25]:
!pip install tensorflow

                                              0.0/300.9 MB ? eta -:--:--
                                              0.3/300.9 MB 8.3 MB/s eta 0:00:37
                                             0.8/300.9 MB 10.1 MB/s eta 0:00:30
                                             1.3/300.9 MB 11.9 MB/s eta 0:00:26
                                             1.9/300.9 MB 11.7 MB/s eta 0:00:26
                                             2.2/300.9 MB 11.8 MB/s eta 0:00:26
                                             2.8/300.9 MB 11.7 MB/s eta 0:00:26
                                             3.3/300.9 MB 12.2 MB/s eta 0:00:25
                                             3.8/300.9 MB 12.0 MB/s eta 0:00:25
                                             4.3/300.9 MB 11.9 MB/s eta 0:00:25
                                             4.8/300.9 MB 12.3 MB/s eta 0:00:25
                                             5.3/300.9 MB 12.6 MB/s eta 0:00:24
                                             5.

In [36]:
import pandas as pd

In [42]:
df = pd.read_csv(r"D:\NLP\Fake News\WELFake_Dataset.csv")

In [43]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [44]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [45]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [46]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [47]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [48]:
X = df.drop("label", axis=1)

In [49]:
y = df["label"]

In [50]:
X.shape

(71537, 3)

In [51]:
y.shape

(71537,)

In [52]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences  #padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [53]:
voc_size = 5000

In [54]:
messages = X.copy()

In [56]:
messages["title"][1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [57]:
import nltk
import re
from nltk.corpus import stopwords

In [59]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
#Preprocessing the data
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^A-za-z]', ' ', messages.title[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [68]:
corpus[1:5]

['unbeliev obama attorney gener say charlott rioter peac protest home state north carolina [video]',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group']

In [69]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr[:5]

[[1793, 1683, 965, 330, 2732, 199, 796, 174, 3699, 4729, 2622, 1810],
 [4738,
  841,
  3827,
  1810,
  3391,
  3903,
  3562,
  2889,
  1592,
  114,
  1068,
  3677,
  4242,
  1810],
 [594, 2848, 3744, 4873, 2868, 911, 1564, 4247, 2109, 2158, 817, 4951],
 [2710, 3474, 1786, 3419, 3359, 1281, 3501, 1519, 4044, 2840, 3987],
 [3305, 1564, 4468, 256, 3033, 2514, 3437, 4647, 4468]]

In [70]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina [video]'

In [71]:
onehot_repr[1]

[4738,
 841,
 3827,
 1810,
 3391,
 3903,
 3562,
 2889,
 1592,
 114,
 1068,
 3677,
 4242,
 1810]

In [75]:
length = []
for i in onehot_repr:
    length.append(len(i))

In [77]:
max(length)

47

## Embedding

In [79]:
#Padding to make sentences of same size.
sent_length = 50
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)

## LSTM Model Creation

In [86]:
##creating the model
embedding_vector_features = 50 #hyperparameter  -- feature representation -- each word is represented in this vector of 50
# so sent_length = 50, embedding_vect_feat = 50, so input data would be 50x50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [87]:
len(embedded_docs),y.shape

(71537, (71537,))

In [88]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [90]:
X_final.shape, y_final.shape

((71537, 50), (71537,))

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final, test_size=0.33, random_state=42)

In [94]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10, batch_size=64)

Epoch 1/10
749/749 [==============================] - 35s 47ms/step - loss: 0.2180 - accuracy: 0.9112 - val_loss: 0.2476 - val_accuracy: 0.8983
Epoch 2/10
749/749 [==============================] - 32s 42ms/step - loss: 0.1833 - accuracy: 0.9260 - val_loss: 0.2528 - val_accuracy: 0.8939
Epoch 3/10
749/749 [==============================] - 35s 47ms/step - loss: 0.1567 - accuracy: 0.9386 - val_loss: 0.2785 - val_accuracy: 0.8941
Epoch 4/10
749/749 [==============================] - 36s 49ms/step - loss: 0.1348 - accuracy: 0.9465 - val_loss: 0.2965 - val_accuracy: 0.8917
Epoch 5/10
749/749 [==============================] - 34s 45ms/step - loss: 0.1156 - accuracy: 0.9538 - val_loss: 0.3209 - val_accuracy: 0.8889
Epoch 6/10
749/749 [==============================] - 44s 58ms/step - loss: 0.0958 - accuracy: 0.9623 - val_loss: 0.3937 - val_accuracy: 0.8900
Epoch 7/10
749/749 [==============================] - 43s 57ms/step - loss: 0.0811 - accuracy: 0.9678 - val_loss: 0.4628 - val_accuracy:

In [95]:
y_pred=model.predict(X_test)

738/738 [==============================] - 13s 16ms/step


In [96]:
y_pred=np.where(y_pred > 0.5, 1,0)  #right now we chose probablility as 0.5, usually u choose looking at ROC_AUC

In [97]:
from sklearn.metrics import confusion_matrix

In [98]:
confusion_matrix(y_test,y_pred)

array([[10281,  1412],
       [ 1192, 10723]], dtype=int64)

In [99]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8896984073195527

In [100]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89     11693
           1       0.88      0.90      0.89     11915

    accuracy                           0.89     23608
   macro avg       0.89      0.89      0.89     23608
weighted avg       0.89      0.89      0.89     23608

